In [1]:
%load_ext autoreload
%autoreload 2
from src.data_processing import load_data
import itertools
import string

In [2]:
from src.ngrams import *
from src.string_similarity import levenshtein
import operator
from src.data_processing import print_progress
from nltk import word_tokenize, pos_tag
from src.data_processing import load_cmu
from listener.ipatoarpabet import translate
from string import punctuation

In [3]:
task1, task2, task3, min_pairs = load_data()
cmu = load_cmu()

In [4]:
ngram_frequencies = json.load(open("data/ngram_frequencies.json"))
task2_frequencies = [n for i, n in enumerate(ngram_frequencies) if task1[i]['pun']]

In [5]:
test_freqencies = [{'frequencies': t2} for t2 in task2_frequencies]
for i, t in enumerate(task2):
    test_freqencies[i]['text'] = t['words']
    test_freqencies[i]['target'] = t['target']

In [6]:
with open("data/ngram_search_space.json") as f:
    ngram_search_space = json.load(f)
    

In [7]:
search_space = []
frequencies = []
for i in range(len(ngram_search_space)):
    if 'Tom' not in test_freqencies[i]['text']:
        search_space.append(ngram_search_space[i])
        frequencies.append(test_freqencies[i])
        

In [9]:
def switch_score(distance, frequency_difference):
    return frequency_difference / ((distance + 1) ** 2)

In [15]:
def find_target(index):
    print(index)
    result = []
    search = search_space[index]
    for candidate, other_ngrams in search:
        print(candidate, other_ngrams)
        try:
            if candidate[0] in cmu:
                cand_phone = cmu[candidate[0].lower()][0]
            else:
                cand_phone = translate(candidate[0])[0].split()
        except:
            print("continuing")
            continue
        
        # skipping due to huge search space...
        if len(other_ngrams) > 10000:
            result.append(("skipped", candidate[0], 0))
            print("skipped at {}".format(i))
            continue
        for ngram, freq in other_ngrams.items():

            switch_word = ngram.split()[1]
            print(switch_word)
            if any(w in punctuation for w in switch_word):
                continue
            try:
                if switch_word in cmu:
                    switch_phone = cmu[switch_word][0]
                else:
                    switch_phone = translate(switch_word)[0].split()
            except:
                continue
            l_distance = levenshtein(cand_phone, switch_phone)
            
            if l_distance <= 2:
                result.append((switch_word, candidate[0], l_distance, freq))
                              
    return list(sorted(result, key=operator.itemgetter(2), reverse=True))

In [17]:
find_target(81)

81
['sinners', 8, 0] {'more direction can': 96, 'more at can': 447, 'more baby can': 49, 'more Canadians can': 172, 'more rewards can': 47, 'more indices can': 76, 'more enjoy can': 41, 'more troops can': 146, 'more study can': 57, 'more media can': 64, 'more cooperation can': 54, 'more authentic can': 47, 'more personal can': 209, 'more healthfully can': 65, 'more administrators can': 129, 'more routers can': 59, 'more reports can': 106, 'more buildings can': 98, 'more expected can': 119, 'more restrictions can': 65, 'more clarity can': 73, 'more adventure can': 88, 'more attributes can': 226, 'more quotes can': 86, 'more patterns can': 256, 'more hypocritical can': 45, 'more streams can': 81, 'more life can': 137, 'more diversion can': 49, 'more machines can': 221, 'more carefully can': 88, 'more switches can': 213, 'more advances can': 41, 'more beautiful can': 107, 'more clothes can': 53, 'more calls can': 393, 'more patients can': 1365, 'more sunlight can': 89, 'more violence can'

votes
books
democratic
colours
features
solutions
views
sense
communities
modules
joints
filling
paper
ads
pins
planes
sets
perfect
cynical
issues
speed
values
interest
antennas
alkaline
instructors
delay
charges
experienced
stupid
bridges
alleles
expensive
who
homes
competition
observations
text
traffic
packets
subjects
discussions
videos
knowledgeable
cheaply
questions
phones
floors
stars
relations
really
able
intelligence
readily
positions
movies
expertise
sounds
profits
houses
>
lies
credence
exposure
filters
techniques
informations
goods
photos
posts
chemicals
simply
technologies
also
intense
affluent
problems
reactions
scientists
entrepreneurs
mail
fluid
women
contractors
sessions
qualified
weapons
loans
countries
purchases
competitors
]
eyes
but
tables
reliance
forms
fit
AIDS
clubs
ships
sources
rounds
secure
Christians
fully
Gifts
impact
gain
bugs
activity
luxurious
designs
basic
successful
independent
SMEs
tiles
personally
ink
effective
:
where
assignments
rings
everyone
user


references
artists
gates
digits
paths
Family
shares
investments
interaction
tabs
and
cases
strength
times
packages
varieties
keys
explanation
noise
trade
activities
responsibility
tension
writers
-
components
poeple
branches
proof
knowledge
service
explicit
pilgrims
Canadian
signals
information
['more', 7, 0] {'or " sinners': 152, 'or the sinners': 108, 'or for sinners': 52}
"
the
for
['can', 9, 0] {}


[('pins', 'sinners', 2, 61),
 ('sounds', 'sinners', 2, 96),
 ('cities', 'sinners', 2, 107),
 ('skins', 'sinners', 2, 120),
 ('learners', 'sinners', 2, 89),
 ('servers', 'sinners', 2, 471),
 ('seniors', 'sinners', 2, 110),
 ('sensors', 'sinners', 2, 162),
 ('for', 'more', 1, 52)]

In [11]:
from multiprocessing import Pool
p = Pool(4)
ranked_replacements = p.map(find_target, range(len(search_space)))

66
0
132
198
1
133
134
2
135
67
68
69
70
71
72
73
136
199
200
3
4
201
skipped at 1270
202
203
204
5
skipped at 1270
6
skipped at 1270
7
8
skipped at 1270
9
10
11
74
205
206
skipped at 1270
137
138
12
13
207
208
139
75
76
77
78
140
79
skipped at 1270
80
141
142
14
15
16
17
18
skipped at 1270
19
20
21
209
81
82
skipped at 1270
83
84
22
23
24
25
26
27
28
29
30
skipped at 1270
31
32
143
144
145
skipped at 1270
146
skipped at 1270
147
33
34
35
skipped at 1270
36
85
86
210
87
88
89
90
skipped at 1270
91
92
37
38
39
211
40
93
94
skipped at 1270
95
96
97
212
skipped at 1270
skipped at 1270
213
98
214
99
100
148
101
102
103
skipped at 1270
104
skipped at 1270
105
skipped at 1270
215
41
42
43
44
216
skipped at 1270
149
150
151
152
45
217
skipped at 1270
218
46
106
153
154
155
156
157
158
skipped at 1270
159
160
161
skipped at 1270
162
163
164
165
166
107
108
109
110
111
112
skipped at 1270
219
167
168
169
47
skipped at 1270
48
49
skipped at 1270
220
50
51
52
221
53
222
223
113
114
115
116
117
11

In [14]:
with open("data/ranked_replacements.json", 'w') as f:
    json.dump(ranked_replacements, f, indent=4)